In [1]:
import pandas as pd

In [2]:
# TODO: change to the newly created SNOTEL seasonal_mean.csv
snotel_df_path = '../content/csv/snotel/west_slopes_south/seasonal_mean.csv'
# TODO: change to the newly created NWAC windspeeds.csv
nwac_df_path = '../content/csv/nwac/west_slopes_south/windspeeds.csv'

danger_df_path = '../content/csv/danger_ratings_by_elevation.csv'

# TODO: change to the area
danger_area = 'West Slopes South'

# TODO: change output file to be named correctly
output_file = '../content/csv/FINALDATASET_WestSlopesSouth.csv'

def load_df(path):
    df = pd.read_csv(path)
    return df

def reformat_date(danger_df):
    # (thank you Jason for the stuff in dates1.ipynb, I really needed that reference)
    danger_df['Date'] = pd.to_datetime(danger_df['Date']).dt.date

def select_danger_area(danger_df):
    danger_df = danger_df[danger_df['Area'] == danger_area]
    return danger_df

def map_danger_to_number(danger_df):
    danger_df['Above Treeline'] = danger_df['Above Treeline'].map(
        { 'NO RATING': 0, 'LOW': 1, 'MODERATE': 2, 'CONSIDERABLE': 3, 'HIGH': 4, 'EXTREME': 5 })
    danger_df['At Treeline'] = danger_df['At Treeline'].map(
        { 'NO RATING': 0, 'LOW': 1, 'MODERATE': 2, 'CONSIDERABLE': 3, 'HIGH': 4, 'EXTREME': 5 })
    danger_df['Below Treeline'] = danger_df['Below Treeline'].map(
        { 'NO RATING': 0, 'LOW': 1, 'MODERATE': 2, 'CONSIDERABLE': 3, 'HIGH': 4, 'EXTREME': 5 })
    return danger_df

def set_date_as_index(df):
    df.set_index('Date', inplace=True)

snotel_df = load_df(snotel_df_path)
nwac_df = load_df(nwac_df_path)
danger_df = load_df(danger_df_path)

danger_df = select_danger_area(danger_df)
danger_df = map_danger_to_number(danger_df)

reformat_date(snotel_df)
reformat_date(nwac_df)
reformat_date(danger_df)

set_date_as_index(snotel_df)
set_date_as_index(nwac_df)
set_date_as_index(danger_df)

In [3]:
# Make sure df loaded correctly
snotel_df.head()

,Max Air Temp 24hr,Total Snowfall 24hr,Delta SWE 24hr,Min Temp Diff 48hr
Date,,,,
2022-11-01,33.875,0.50,-0.100,-3.775
2022-11-02,33.275,0.75,-0.025,-3.975
2022-11-03,33.125,1.50,0.100,0.050
2022-11-04,40.725,4.50,1.125,7.600
2022-11-05,34.000,-3.75,-0.125,-4.675


In [4]:
# Make sure df loaded correctly
nwac_df.tail()

,Max Windspeed 24hr,Was High Winds 24hr
Date,,
2023-05-25,27.296667,1
2023-05-26,20.946667,0
2023-05-27,23.650000,0
2023-05-28,22.945000,0
2023-05-29,21.535000,0


In [5]:
# Make sure df loaded correctly
danger_df.head()

,Area,Above Treeline,At Treeline,Below Treeline
Date,,,,
2023-04-15,West Slopes South,2,2,1
2023-04-14,West Slopes South,1,1,1
2023-04-13,West Slopes South,2,2,1
2023-04-12,West Slopes South,2,2,1
2023-04-11,West Slopes South,2,2,2


In [6]:
def combine_dfs(snotel_df, nwac_df, danger_df):
    return danger_df.join(snotel_df).join(nwac_df)

combined_df = combine_dfs(snotel_df, nwac_df, danger_df)
combined_df.head()

,Area,Above Treeline,At Treeline,Below Treeline,Max Air Temp 24hr,Total Snowfall 24hr,Delta SWE 24hr,Min Temp Diff 48hr,Max Windspeed 24hr,Was High Winds 24hr
Date,,,,,,,,,,
2022-11-25,West Slopes South,1,1,1,43.150,0.50,0.025,-5.200,43.066667,1
2022-11-26,West Slopes South,3,2,1,35.800,1.25,0.150,-0.975,42.596667,1
2022-11-27,West Slopes South,2,2,1,32.325,-1.00,0.125,-0.975,57.423333,1
2022-11-28,West Slopes South,2,2,1,27.225,6.50,0.925,-5.925,77.890000,1
2022-11-29,West Slopes South,3,3,2,28.950,2.75,0.250,0.650,38.360000,1


In [7]:
# Killer. now we can calculate those extra columns

def take_snotel_df_and_calculate_columns(df):
    """Using raw SNOTEL data, calculate the above columns and return them as a dataframe"""

    # Add the column placeholders to the dataframe (or initialize a new one); initialize as null for now
    # String shorthand
    max_temp_72 = 'Max Air Temp 72hr'
    tot_snow_72 = 'Total Snowfall 72hr'
    wgt_snow_72 = 'Weighted Snowfall 96hr'
    was_hvys_24 = 'Was Heavy Snowfall 24hr'
    sum_mtmp_72 = 'Sum Max Temp 72hr'
    df[max_temp_72] = 0 # None
    df[tot_snow_72] = 0 # None
    df[wgt_snow_72] = 0 # None
    df[was_hvys_24] = 0 # None
    df[sum_mtmp_72] = 0 # None

    # Calculate the above metrics
    one_day_ago = None # This wouldn't be necessary if I wasn't indexing on date
    two_days_ago = None
    three_days_ago = None
    for idx, day in df.iterrows():
        df.at[idx, was_hvys_24] = 1 if day['Total Snowfall 24hr'] >= 12 else 0 # todo: update threshold
        if two_days_ago is not None:
            df.at[idx, max_temp_72] = max(
                day['Max Air Temp 24hr'], 
                one_day_ago['Max Air Temp 24hr'], 
                two_days_ago['Max Air Temp 24hr'])
            df.at[idx, tot_snow_72] = \
                day['Total Snowfall 24hr'] \
                + one_day_ago['Total Snowfall 24hr'] \
                + two_days_ago['Total Snowfall 24hr']
            df.at[idx, sum_mtmp_72] = \
                day['Max Air Temp 24hr'] \
                + one_day_ago['Max Air Temp 24hr'] \
                + two_days_ago['Max Air Temp 24hr'] 
        if three_days_ago is not None:
            df.at[idx, 'Weighted Snowfall 96hr'] = \
                day['Total Snowfall 24hr'] * 1.0 \
                + one_day_ago['Total Snowfall 24hr'] * 0.75 \
                + two_days_ago['Total Snowfall 24hr'] * 0.5 \
                + three_days_ago['Total Snowfall 24hr'] * 0.25

        # Update past day placeholders
        three_days_ago = two_days_ago
        two_days_ago = one_day_ago
        one_day_ago = day
    
    # Return the new/updated dataframe
    return df

calculated_columns = take_snotel_df_and_calculate_columns(combined_df)
calculated_columns.head()

,Area,Above Treeline,At Treeline,Below Treeline,Max Air Temp 24hr,Total Snowfall 24hr,Delta SWE 24hr,Min Temp Diff 48hr,Max Windspeed 24hr,Was High Winds 24hr,Max Air Temp 72hr,Total Snowfall 72hr,Weighted Snowfall 96hr,Was Heavy Snowfall 24hr,Sum Max Temp 72hr
Date,,,,,,,,,,,,,,,
2022-11-25,West Slopes South,1,1,1,43.150,0.50,0.025,-5.200,43.066667,1,0.000,0.00,0.0000,0,0.000
2022-11-26,West Slopes South,3,2,1,35.800,1.25,0.150,-0.975,42.596667,1,0.000,0.00,0.0000,0,0.000
2022-11-27,West Slopes South,2,2,1,32.325,-1.00,0.125,-0.975,57.423333,1,43.150,0.75,0.0000,0,111.275
2022-11-28,West Slopes South,2,2,1,27.225,6.50,0.925,-5.925,77.890000,1,35.800,6.75,6.5000,0,95.350
2022-11-29,West Slopes South,3,3,2,28.950,2.75,0.250,0.650,38.360000,1,32.325,8.25,7.4375,0,88.500


In [8]:
def add_yesterday_danger(df):
    df.insert(4, 'Yesterday Danger Above Treeline', 0)
    df.insert(5, 'Yesterday Danger At Treeline', 0)
    df.insert(6, 'Yesterday Danger Below Treeline', 0)
    yest = None
    for idx, day in df.iterrows():
        if yest is not None:
            df.at[idx, 'Yesterday Danger Above Treeline'] = yest['Above Treeline']
            df.at[idx, 'Yesterday Danger At Treeline'] = yest['At Treeline']
            df.at[idx, 'Yesterday Danger Below Treeline'] = yest['Below Treeline']
        yest = day
    return df

yesterday_danger = add_yesterday_danger(calculated_columns)
yesterday_danger.head()

,Area,Above Treeline,At Treeline,Below Treeline,Yesterday Danger Above Treeline,Yesterday Danger At Treeline,Yesterday Danger Below Treeline,Max Air Temp 24hr,Total Snowfall 24hr,Delta SWE 24hr,Min Temp Diff 48hr,Max Windspeed 24hr,Was High Winds 24hr,Max Air Temp 72hr,Total Snowfall 72hr,Weighted Snowfall 96hr,Was Heavy Snowfall 24hr,Sum Max Temp 72hr
Date,,,,,,,,,,,,,,,,,,
2022-11-25,West Slopes South,1,1,1,0,0,0,43.150,0.50,0.025,-5.200,43.066667,1,0.000,0.00,0.0000,0,0.000
2022-11-26,West Slopes South,3,2,1,1,1,1,35.800,1.25,0.150,-0.975,42.596667,1,0.000,0.00,0.0000,0,0.000
2022-11-27,West Slopes South,2,2,1,3,2,1,32.325,-1.00,0.125,-0.975,57.423333,1,43.150,0.75,0.0000,0,111.275
2022-11-28,West Slopes South,2,2,1,2,2,1,27.225,6.50,0.925,-5.925,77.890000,1,35.800,6.75,6.5000,0,95.350
2022-11-29,West Slopes South,3,3,2,2,2,1,28.950,2.75,0.250,0.650,38.360000,1,32.325,8.25,7.4375,0,88.500


In [9]:
yesterday_danger.to_csv(output_file)